In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN_Wmultitask_8_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [7]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [2, 2, 2]
weights_shape = [[5, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
activation = tf.nn.relu
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)


variables_weights = np.ones((n_classes,))

In [9]:
pos_weight.shape

(19,)

In [10]:
pos_weight

array([  0.40793684,  51.83632019,  50.54312354,  53.39606396,
        53.49661121,  43.82919412,  45.64978903,  64.85852569,
        69.47649402,  84.21001927,  98.26823793,  89.34525026,
        99.39500568,  75.77777778,  76.17975567,  46.17226667,
        13.60019809,  65.60240964,  20.4471387 ])

In [11]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [12]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/' + exp_name 

In [13]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [14]:
model.build_conv_net(pos_weight_values = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation,
                     trainable_conv = True,
                     trainable_last = True,
                     variables_weights = variables_weights)

In [ ]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [ ]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.059 vs. 0.058
F1 = 0.088 vs. 0.086
Accuracy = 0.009 vs. 0.056
CHANGE DETECTION
AUC = 0.053 vs. 0.049
F1 = 0.094 vs. 0.094
Precision = 0.055 vs. 0.050
Recall = 0.331 vs. 0.837
Validation
ACTIVITY RECOGNITION
AUC = 0.058 vs. 0.056
F1 = 0.081 vs. 0.074
Accuracy = 0.008 vs. 0.056
CHANGE DETECTION
AUC = 0.036 vs. 0.026
F1 = 0.078 vs. 0.051
Precision = 0.048 vs. 0.028
Recall = 0.206 vs. 0.263
Minibatch Loss= 1.384734
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.478 vs. 0.059
F1 = 0.519 vs. 0.084
Accuracy = 0.664 vs. 0.056
CHANGE DETECTION
AUC = 0.194 vs. 0.054
F1 = 0.278 vs. 0.088
Precision = 0.244 vs. 0.051
Recall = 0.323 vs. 0.317
Validation
ACTIVITY RECOGNITION
AUC = 0.318 vs. 0.058
F1 = 0.383 vs. 0.077
Accuracy = 0.667 vs. 0.056
CHANGE DETECTION
AUC = 0.100 vs. 0.026
F1 = 0.181 vs. 0.050
Precision = 0.128 vs. 0.026
Recall = 0.312 vs. 0.799
Minibatch Loss= 0.438777


In [23]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model/CNN_Wmultitask_8_opp/CNN_Wmultitask_8_opp-20000'

In [24]:
#load the model
#saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
#saver.restore(sess, '../model/CNN_activity_multitask_64_opp/CNN_activity_multitask_64_opp-20000')


In [25]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.584 vs. 0.056
F1 = 0.602 vs. 0.074
Accuracy = 0.840 vs. 0.056
CHANGE DETECTION
AUC = 0.117 vs. 0.026
F1 = 0.210 vs. 0.053
Precision = 0.180 vs. 0.039
Recall = 0.253 vs. 0.082


In [26]:
out = nn_eval.deploy (model, sess, test_data, test_gt)

In [27]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(probability_of_pos, axis = 2)

In [28]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})

In [29]:
1

1